In [24]:
import numpy as np
import pandas as pd
import os

# Brainstorm Ideas:

784 inputs, 2 hidden layers, 0-9 as output

In [25]:
def sigmoid(x):
  # Our activation function: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

In [26]:
dataframe = pd.read_csv('./digit-recognizer/train.csv')
PIXEL_COUNT = len(dataframe.columns) - 1
SAMPLE_SIZE = len(dataframe)

In [27]:
def deriv_sigmoid(x):
    # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
    fx = sigmoid(x)
    return fx * (1 - fx)

In [28]:
class Neuron:
    def __init__(self, weights, bias, value=0):
        self.weights = weights # weights type: numpy array
        self.bias = bias # bias type: integer
        self.value = value
    
    def weighted_output(self, inputs):
        # feed forward algorithm
        # inputs: original image input / weights
        output_node = np.dot(inputs, self.weights) + self.bias
        return sigmoid(output_node)  

In [39]:
class NeuralNetwork:
    def __init__(self, hidden_layers, input_layer):
        # hidden_layer: list of number of nodes in the hidden layer
        
        self.network = [[]]
        # initialize input layer (zeroth layer)
        for i in range(PIXEL_COUNT):
            weights = np.random.uniform(low=0.0, high=1.0, size=PIXEL_COUNT)
            bias = np.random.normal()
            neuron = Neuron(weights, bias, input_layer)
            self.network[0].append(neuron)
        
        # initialize hidden layers
        for layer in hidden_layers:
            # append a list that contains all neurons in this hidden layer
            self.network.append([])
            for n in range(layer):
                weights = np.random.uniform(low=0.0, high=1.0, size=PIXEL_COUNT)
                bias = np.random.normal()
                neuron = Neuron(weights, bias)
                self.network[-1].append(neuron)
                
        # initialize output layer
        self.network.append([])
        for i in range(10):
            weights = np.random.uniform(low=0.0, high=1.0, size=PIXEL_COUNT)
            bias = np.random.normal()
            neuron = Neuron(weights, bias)
            self.network[-1].append(neuron)
        
        
    def print_network(self):
        print(len(self.network))

In [41]:
digit_recognition_network = NeuralNetwork([28, 28], dataframe.iloc[1][1:])
digit_recognition_network.print_network()

4
